# Análise Completa de Desempenho e Profiling

Este notebook apresenta uma análise detalhada do desempenho dos algoritmos Hill Climbing (HC), Simulated Annealing (SA) e várias configurações de Algoritmos Genéticos (AG) para o problema de atribuição de jogadores a equipas desportivas. Inclui uma análise dos tempos de execução e dos dados de profiling obtidos com `cProfile` para identificar gargalos de desempenho e sugerir estratégias de otimização.

## 1. Configuração do Experimento

- **Dataset:** `players.csv` (35 jogadores)
- **Número de Equipas:** 5
- **Tamanho da Equipa:** 7 jogadores
- **Orçamento Máximo por Equipa:** 750M €
- **Número de Execuções por Algoritmo:** 1
- **Algoritmos Genéticos (AGs):** 1 geração para estimativa de tempo
- **Profiling:** Ativado com `cProfile` para toda a execução do script `main_script_sp_timing_estimate.py`.

## 2. Resultados de Tempo de Execução e Fitness

Os seguintes tempos de execução e valores de fitness foram registados durante a execução completa do script.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Dados resumidos da execução completa (baseados no output do script)
# Estes dados devem ser extraídos do ficheiro CSV gerado pelo script ou do seu output
# Por agora, vamos usar os valores que vimos no output do terminal para HC e SA,
# e adicionar estimativas para os AGs baseadas na sua execução (que foi rápida por ser 1 geração)
results_data = {
    'Algorithm': [
        'Hill Climbing (SP-Timing)', 
        'Simulated Annealing (SP-Timing)',
        'GA_Cfg1_AdSwapC_Ad1PtPV_SelTournK3 (SP-Timing)',
        'GA_Cfg2_AdTargEx_AdUnifPV_SelRank (SP-Timing)',
        'GA_Cfg3_AdShufWTC_Ad1PtPV_SelBoltz (SP-Timing)',
        'GA_Cfg4_AdTargEx_AdUnifPV_SelTournK5 (SP-Timing)',
        'GA_Cfg5_BaseSwap_Base1Pt_SelTournK4 (SP-Timing)',
        'GA_Cfg6_BaseTeamShift_AdUnifPV_SelTournK3 (SP-Timing)',
        'GA_Cfg7_BaseShuffle_Ad1PtPV_SelBoltz (SP-Timing)',
        'GA_Cfg8_BaseShuffle_BaseUnif_SelTournK4 (SP-Timing)',
        'GA_Cfg9_AdTargEx_Base1Pt_SelRank (SP-Timing)'
    ], 
    'Mean Execution Time (s)': [
        0.62,  # HC, valor da última execução bem sucedida
        73.17, # SA, valor da última execução bem sucedida (simulated_annealing function time)
        0.14,  # Estimativa baseada no output do CSV para GA_Cfg1
        0.28,  # Estimativa baseada no output do CSV para GA_Cfg2
        1.06,  # Estimativa baseada no output do CSV para GA_Cfg3
        0.21,  # Estimativa baseada no output do CSV para GA_Cfg4
        0.17,  # Estimativa baseada no output do CSV para GA_Cfg5
        0.17,  # Estimativa baseada no output do CSV para GA_Cfg6
        1.07,  # Estimativa baseada no output do CSV para GA_Cfg7
        0.14,  # Estimativa baseada no output do CSV para GA_Cfg8
        1.07   # Estimativa baseada no output do CSV para GA_Cfg9
    ], 
    'Overall Best Fitness': [
        0.0571, 0.0571, 0.2914, 0.2770, 0.3876, 0.3307, 0.1723, 0.3051, 0.2770, 0.6478, 0.2914 # Valores do CSV
    ]
}
df_results = pd.DataFrame(results_data)

print("Resumo dos Resultados de Desempenho:")
print(df_results.to_string())

# Gráfico comparativo de tempos de execução
plt.figure(figsize=(12, 8))
bars = plt.bar(df_results['Algorithm'], df_results['Mean Execution Time (s)'], color='skyblue')
plt.xlabel('Algoritmo')
plt.ylabel('Tempo Médio de Execução (s) - Escala Log')
plt.title('Comparação de Tempo de Execução dos Algoritmos (1 execução, AGs 1 geração)')
plt.xticks(rotation=90)
plt.yscale('log') # Escala logarítmica devido à grande variação
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, f'{yval:.2f}s', va='bottom', ha='center')
plt.tight_layout()
plt.show()

## 3. Análise Detalhada do Profiling com `cProfile`

O script `main_script_sp_timing_estimate.py` foi executado com `cProfile` para recolher dados detalhados sobre o tempo gasto em cada função. O ficheiro de resultados do profiling (`timing_estimate_profile.prof`) foi analisado usando o script `analyze_profile.py`. Abaixo, apresentamos os principais resultados dessa análise.

In [ ]:
# Ler e apresentar o conteúdo do ficheiro de análise de profiling
analysis_file_path = "/home/ubuntu/CIFO_EXTENDED_Project/images_sp/timing_estimate/profiling_results/profile_analysis.txt"

try:
    with open(analysis_file_path, 'r') as f:
        profile_analysis_content = f.read()
    print("Análise do Profiling (do ficheiro profile_analysis.txt):
")
    print(profile_analysis_content)
except FileNotFoundError:
    print(f"Erro: O ficheiro {analysis_file_path} não foi encontrado.")

### 3.1. Principais Observações do Profiling:

A análise do ficheiro `profile_analysis.txt` (que contém o output de `pstats`) revela os seguintes pontos críticos:

1.  **Tempo Total do Script:** A execução completa do script demorou aproximadamente **85.66 segundos**.
2.  **Dominância do Simulated Annealing (SA):** A função `simulated_annealing` (em `evolution.py`) é, de longe, a que mais consome tempo, com **73.17 segundos** de tempo cumulativo. Isto representa cerca de 85% do tempo total de execução do script.
3.  **`copy.deepcopy` como Principal Gargalo:** Dentro do SA (e potencialmente noutras partes, mas maioritariamente no SA), a função `copy.deepcopy` é o maior consumidor de tempo. Foi chamada mais de 23 milhões de vezes, totalizando **53.49 segundos** de tempo cumulativo. As suas sub-funções `_deepcopy_list` e `_deepcopy_dict` também figuram proeminentemente.
4.  **Funções da Solução no SA:**
    *   `solution.py:178(get_random_neighbor)`: Chamada 45,850 vezes, consumiu **19.06 segundos** de tempo cumulativo. Esta é a função específica do SA para gerar vizinhos.
    *   `solution.py:122(fitness)`: Chamada 73,855 vezes, consumiu **16.03 segundos** de tempo cumulativo.
    *   `solution.py:85(is_valid)`: Chamada 200,249 vezes, consumiu **10.74 segundos** de tempo cumulativo.
5.  **Desempenho dos Algoritmos Genéticos (AGs):** Os AGs, com apenas 1 geração, foram relativamente rápidos. A função `genetic_algorithm` (em `evolution.py`) foi chamada 9 vezes (uma para cada configuração) e consumiu um total de **5.88 segundos** de tempo cumulativo. As funções `is_valid` e `fitness` também são chamadas dentro dos AGs, mas o seu impacto é menor devido ao número reduzido de gerações.
6.  **Desempenho do Hill Climbing (HC):** A função `hill_climbing` (em `evolution.py`) foi muito rápida, consumindo apenas **0.62 segundos** de tempo cumulativo.

Estes dados confirmam que o **Simulated Annealing é o principal alvo para otimização**, e que a operação `deepcopy` dentro dele é o maior contribuinte para a sua lentidão.

## 4. Estratégias de Otimização Sugeridas (Foco no SA)

Com base na análise de profiling, as seguintes estratégias são sugeridas para otimizar o Simulated Annealing:

1.  **Reduzir ou Eliminar `deepcopy`:**
    *   **Analisar a necessidade:** Verificar cada chamada a `deepcopy` dentro do `simulated_annealing` e `get_random_neighbor`. É sempre necessário uma cópia completa? 
    *   **Cópia Seletiva/Incremental:** Em vez de copiar toda a solução, talvez seja possível copiar apenas as partes que mudam ou aplicar/reverter alterações de forma incremental. Por exemplo, se `get_random_neighbor` apenas troca dois jogadores, podemos aplicar a troca, avaliar, e depois revertê-la se a nova solução não for aceite, em vez de copiar a solução inteira a cada vez.
    *   Usar `copy.copy()` (cópia superficial) se uma cópia profunda não for estritamente necessária, mas isto requer cuidado para evitar efeitos secundários com objetos mutáveis partilhados.

2.  **Otimizar `get_random_neighbor`:**
    *   Esta função já consome bastante tempo. Assegurar que a lógica de geração e validação de um vizinho aleatório é o mais eficiente possível. Se a validação (`is_valid`) for chamada muitas vezes aqui, otimizar `is_valid` também ajudará.

3.  **Otimizar `is_valid()` e `fitness()`:**
    *   **Vetorização com NumPy:** Como discutido anteriormente, converter partes destas funções para usar operações vetorizadas do NumPy em vez de loops Python explícitos pode trazer ganhos significativos. Por exemplo, contar jogadores por equipa, verificar restrições posicionais, e calcular somas/médias de custos/skills.
    *   **Cálculo Incremental:** Para a função `fitness` (e potencialmente `is_valid`), se uma alteração de vizinho é pequena (ex: troca de dois jogadores), talvez seja possível recalcular o fitness/validade de forma incremental em vez de o fazer do zero para toda a solução.

4.  **Ajustar Parâmetros do SA:**
    *   Reduzir `iterations_per_temp`: O valor atual é 100. Testar valores menores (ex: 50, 25) para ver o impacto no tempo de execução vs. qualidade da solução.
    *   Acelerar o arrefecimento (diminuir `alpha` de 0.99 para, por exemplo, 0.95 ou 0.90) pode reduzir o número total de iterações.

A prioridade deveria ser dada à questão do `deepcopy`, seguida pela otimização de `is_valid` e `fitness` (potencialmente com NumPy), e depois `get_random_neighbor`.

## 5. Conclusões Finais e Próximos Passos

A execução completa do script de profiling forneceu informações cruciais sobre os gargalos de desempenho. O Simulated Annealing é o algoritmo mais lento, principalmente devido ao uso extensivo de `deepcopy` e ao elevado número de chamadas às funções `is_valid` e `fitness`.

Próximos passos recomendados:
1.  **Implementar as otimizações sugeridas**, começando pelas que prometem maior impacto (ex: `deepcopy` e vetorização de `is_valid`/`fitness`).
2.  **Executar novamente o script de profiling** após cada otimização significativa para medir o impacto.
3.  **Comparar a qualidade das soluções** obtidas após as otimizações para garantir que a aceleração não comprometeu a capacidade dos algoritmos de encontrar boas soluções.